In [1]:
import cv2
import pandas as pd
import numpy as np
import torch
import ultralytics
from ultralytics import YOLO
from tracker import*

In [2]:
model=YOLO('yolov8n.pt')

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

cpu


In [3]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [4]:
tracker=Tracker()
video=cv2.VideoCapture('road_feed.mp4')

In [5]:
counter = []

In [6]:
while True:    
  ret,frame = video.read()
  if not ret:
    break
  
  frame=cv2.resize(frame,(540, 960))
  
  results=model.predict(frame)
  temp=results[0].boxes.data
  temp = temp.detach().cpu().numpy()
  bb_df=pd.DataFrame(temp).astype("int")
  # print(bb_df)

  bb_list=[]
            
  for index,row in bb_df.iterrows():
    # print(row)
    x1, y1, x2, y2, sc, d = row
    c=class_list[d]
    if c == 'car':
      bb_list.append([x1,y1,x2,y2, sc, c])
      # print(c)
      

  bbox_ids=tracker.update(bb_list)
  #print(bbox_id)
  for bbox in bbox_ids:
      x3,y3,x4,y4, sc, c,id=bbox
      cx=int(x3+x4)//2
      cy=int(y3+y4)//2
      
      # cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
      cv2.putText(frame, str(c), (x3, y3-2), cv2.QT_FONT_NORMAL, 0.3, (255, 255, 255), 1)
      cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 1)
      # cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)



      
      counter_line_y = 480
      offset = 5

      cv2.line(frame, (150, 480), (450, 480), (0, 255, 0), 3)
      cv2.putText(frame, 'counter line', (150, 480-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

      


      if counter_line_y < (cy + offset) and counter_line_y > (cy - offset): 
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),1)
        
        if id not in counter:
          counter.append(id)


  cv2.putText(frame,('count - ')+ str(len(counter)),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)    

  

  cv2.imshow("frames", frame)
  if cv2.waitKey(1)&0xFF==27:
    break
video.release()
cv2.destroyAllWindows()


0: 640x384 10 cars, 231.8ms
Speed: 9.9ms preprocess, 231.8ms inference, 14.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 10 cars, 243.4ms
Speed: 4.3ms preprocess, 243.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 10 cars, 125.6ms
Speed: 4.6ms preprocess, 125.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 9 cars, 138.4ms
Speed: 6.1ms preprocess, 138.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 9 cars, 2 trucks, 132.8ms
Speed: 6.6ms preprocess, 132.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 9 cars, 212.3ms
Speed: 5.9ms preprocess, 212.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 8 cars, 1 bus, 1178.5ms
Speed: 4.3ms preprocess, 1178.5ms inference, 5.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 8 cars, 1032.4ms
Speed: 36.1ms preprocess, 1032.4ms inference, 6.3ms postprocess pe